<a href="https://colab.research.google.com/github/santos-luciano/GTDL/blob/main/Scrapping_do_site_Fundamentus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nos dias atuais muito se diz que devemos investir, mas investir aonde?
Antes de aportamos nosso capital em algum lugar, é de bastante relevância realizar estudos de para onde está indo nosso dinheiro. Quando investimos em uma empresa, estamos buscando a prosperidade tanto individual quanto da empresa. Então é de bastante importância procuramos as melhores empresasas para destinarmos o nosso dinheiro. O que diz se uma empresa é boa? Existem diversos aspectos que podem definir se uma empresa boa,como balanços financeiros, administração, indicadores fundamentalistas e etc. Este último será buscado através do scraping da pagina Fundamentus

#Importando pacotes

In [2]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

# Lendo a página para criar o data.frame

In [3]:
headers = {
    'Accept': '*/*',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-type': 'application/x-www-form-urlencoded',
    'Origin': 'https://www.fundamentus.com.br',
    'Referer': 'https://www.fundamentus.com.br/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Chromium";v="110", "Not A(Brand";v="24", "Google Chrome";v="110"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

page = requests.get('https://www.fundamentus.com.br/resultado.php?segmento=1',
                    headers = headers)


soup_start = BeautifulSoup(page.content,'html.parser')
columns = soup_start.find('tr').text.split('\n')
columns.append('segmento')
papel_fundamentus = pd.DataFrame(columns = columns)


Após a leitura da pagina, temos a tabela abaixo que será preenchida com os ativos financeiros disponiveis no site

In [4]:
papel_fundamentus

,,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,...,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,,segmento


# Navegando pelos setores

A melhor formar encontrada para conserguirmos coletar as informações do site, foi através dos setores. O programa entra no setor e coleta todos os ativos referentes ao setor

In [5]:
for seg in range(1,100):
    try:
      #lendo pagina do segmento "seg"
        page = requests.get('https://www.fundamentus.com.br/resultado.php?segmento={}'.format(seg),
                            headers = headers)
        soup = BeautifulSoup(page.content,'html.parser')
        
        
        
        for row in soup.find_all('tr')[1:]:
            row_papel = row.text.split('\n')
            row_papel.append(seg)
            # concatenando
            papel_fundamentus = pd.concat([papel_fundamentus,
                                           pd.DataFrame([row_papel],
                                                        columns = columns)])
        papel_fundamentus.reset_index(inplace = True,drop=True)
    except:
        pass
    else:
        row_papel = []


O código entra em possíveis 100 setores e concatena até obtermos um data.frame final

# Alterações no banco de dados

Como quase todo banco de dados, foi necessário realizar alterações para que se possa trabalhar com ele

In [ ]:
for col in columns:
    try:
        # tirar os pontos de milhar
        papel_fundamentus[col] = papel_fundamentus[col].str.replace('.','')
        # trocar virgular por ponto e transformar para numero decimal
        papel_fundamentus[col] = papel_fundamentus[col].str.replace(',','.')
        # tirar as porcentagens
        papel_fundamentus[col] = papel_fundamentus[col].str.replace('%','').astype('float')/100

    except:
        pass

# remover colunas vazias
papel_fundamentus = papel_fundamentus.drop(columns = [''])

# Construção de um ranking

Fazendo um ranking baseado nesse artigo https://fiis.com.br/artigos/indicadores-fundos-imobiliarios/. O artigo destaca alguns aspectos importantes que os fundo imobiliarios devem apresentar, como a Liquidez do fundo, P/VP e o Dividend Yeld

In [29]:
# filtrando para selecionar fundos com bastante liquidez
papel_fundamentus_liq = papel_fundamentus[papel_fundamentus['Liq.2meses']>100000]
#montando o ranking em que será levado em consideração o P/VP  e Div Yield
ranking = pd.DataFrame()
ranking['pos'] = range(1,101)
ranking['P/VP'] = papel_fundamentus_liq[papel_fundamentus_liq['P/VP']>0].sort_values(by = ['P/VP'])['Papel'][:100].values
ranking['Div.Yield'] = papel_fundamentus_liq.sort_values(by = ['Div.Yield'],ascending = False)['Papel'][:100].values

In [30]:
# ordenamos
a = ranking.pivot_table(columns = 'P/VP',values = 'pos')
b = ranking.pivot_table(columns = 'Div.Yield',values = 'pos')
c = pd.concat([a,b])

# ranking final é um mixer das classificações de cada papel para um indicador 
ranking_final = c.dropna(axis = 1).sum()

In [33]:
# o que tem a menor pontuação significa que ficou entre as posições mais baixas, ou seja, as primeiras posições.
ranking_final.sort_values()[:10]

LIGT3     22
MRFG3     22
BRPR3     24
BRSR6     33
MOVI3     38
SAPR11    39
BBAS3     39
MYPK3     42
PETR4     43
CPLE11    48
dtype: int64

**Não é uma recomendação de investimento**